In [1]:
import pandas as pd
import numpy as np

In [17]:
# Libraries for developing a Neural Network
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [2]:
train = pd.read_csv('../../ml-data/cyberattack_detection/2_processed_data/A2_training_data.csv', sep=',')

In [4]:
X_train = train.drop('Normal',1)
Y_train = train['Normal']

In [25]:
X_train.head()

,Duration,TotalPackets,TotalBytes,SourceBytes,Protocol_arp,Protocol_gre,Protocol_icmp,Protocol_igmp,Protocol_ipv6,Protocol_pim,...,Direction_<-,Direction_<->,SrcTypeService_-1.0,SrcTypeService_0.0,SrcTypeService_192.0,DestTypeService_-1.0,DestTypeService_0.0,DestTypeService_1.0,DestTypeService_2.0,DestTypeService_3.0
0,-0.191842,-0.009924,-0.008230,-0.019033,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191760,-0.008454,-0.008001,-0.016594,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.191842,-0.010291,-0.008104,-0.018907,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.185597,-0.009924,-0.008237,-0.018491,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.191842,-0.010291,-0.008134,-0.018965,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
test = pd.read_csv('../../ml-data/cyberattack_detection/2_processed_data/A2_test_data.csv', sep=',')

In [6]:
X_test = test.drop('Normal',1)
Y_test = test['Normal']

In [19]:
selector = SelectKBest(mutual_info_classif, k = 10)
selector.fit(X_train,Y_train)


KeyboardInterrupt: 

In [ ]:
selector.get_support()

In [20]:
# Model arguments
args = dict(x = X_train,
            y = Y_train,
            epochs=10,
            validation_split=0.4,
            verbose=2,
            shuffle=True)

# Layer definition
input_layer = Input(shape=(26,))
hidden_layer_1 = Dense(10, activation='relu',activity_regularizer=regularizers.l1(10e-5))(input_layer)
#hidden_layer_2 = Dense(8, activation='sigmoid')(hidden_layer_1)
hidden_layer_2 = Dense(5, activation='relu')(hidden_layer_1)
#hidden_layer_2 = Dropout(0.3)(hidden_layer_2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)

# Model set up
model.compile(tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                270       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 6         
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Execute training
history = model.fit(**args)

Epoch 1/10
37091/37091 - 69s - loss: 0.5522 - accuracy: 0.7250 - val_loss: 0.6773 - val_accuracy: 0.5753
Epoch 2/10
37091/37091 - 69s - loss: 0.5513 - accuracy: 0.7258 - val_loss: 0.6464 - val_accuracy: 0.5759
Epoch 3/10
37091/37091 - 75s - loss: 0.5539 - accuracy: 0.7257 - val_loss: 0.6842 - val_accuracy: 0.5759
Epoch 4/10
37091/37091 - 68s - loss: 0.5545 - accuracy: 0.7272 - val_loss: 0.6656 - val_accuracy: 0.5770
Epoch 5/10
37091/37091 - 67s - loss: 0.5521 - accuracy: 0.7282 - val_loss: 0.6972 - val_accuracy: 0.5766
Epoch 6/10
37091/37091 - 67s - loss: 0.5525 - accuracy: 0.7282 - val_loss: 0.6523 - val_accuracy: 0.5771
Epoch 7/10
37091/37091 - 67s - loss: 0.5531 - accuracy: 0.7283 - val_loss: 0.6537 - val_accuracy: 0.5768
Epoch 8/10
37091/37091 - 65s - loss: 0.5548 - accuracy: 0.7282 - val_loss: 0.6590 - val_accuracy: 0.5770
Epoch 9/10
37091/37091 - 65s - loss: 0.5573 - accuracy: 0.7280 - val_loss: 0.6617 - val_accuracy: 0.5767
Epoch 10/10
37091/37091 - 65s - loss: 0.5574 - accuracy

In [22]:
predict = model.predict(X_test)

In [24]:
results = model.evaluate(X_test,Y_test)

17950/17950 [==============================] - 21s 1ms/step - loss: 0.7973 - accuracy: 0.3514
